In [1]:
import pandas as pd
import numpy as np
import tensorflow
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [2]:
'''data columns are: Index(['Day', 'Month', 'Year', 'States/UTs', 'Rice', 'Wheat', 'Atta (Wheat)',
       'Gram Dal', 'Tur/Arhar Dal', 'Urad Dal', 'Moong Dal', 'Masoor Dal',
       'Sugar', 'Milk @', 'Groundnut Oil (Packed)', 'Mustard Oil (Packed)',
       'Vanaspati (Packed)', 'Soya Oil (Packed)', 'Sunflower Oil (Packed)',
       'Palm Oil (Packed)', 'Gur', 'Tea Loose', 'Salt Pack (Iodised)',
       'Potato', 'Onion', 'Tomato'],
      dtype='object')'''

"data columns are: Index(['Day', 'Month', 'Year', 'States/UTs', 'Rice', 'Wheat', 'Atta (Wheat)',\n       'Gram Dal', 'Tur/Arhar Dal', 'Urad Dal', 'Moong Dal', 'Masoor Dal',\n       'Sugar', 'Milk @', 'Groundnut Oil (Packed)', 'Mustard Oil (Packed)',\n       'Vanaspati (Packed)', 'Soya Oil (Packed)', 'Sunflower Oil (Packed)',\n       'Palm Oil (Packed)', 'Gur', 'Tea Loose', 'Salt Pack (Iodised)',\n       'Potato', 'Onion', 'Tomato'],\n      dtype='object')"

In [3]:
# Load the data from the CSV file
df = pd.read_csv(r"data\cleansing\filled\data_r_mean_filled.csv")

In [4]:
print(df.shape)

(89280, 26)


In [5]:
df = df[df["Year"] >2023]
print(df.shape)

(8054, 26)


In [6]:
# Convert date columns to a datetime object
df['Date'] = pd.to_datetime(df[['Day', 'Month', 'Year']])
df.set_index('Date', inplace=True)

In [7]:
# Select the crops for prediction
crops = ["Rice", "Wheat", "Atta (Wheat)", "Gram Dal", "Tur/Arhar Dal", "Urad Dal", "Moong Dal", "Masoor Dal", "Sugar", "Milk @", "Groundnut Oil (Packed)", "Mustard Oil (Packed)", "Vanaspati (Packed)", "Soya Oil (Packed)", "Sunflower Oil (Packed)", "Palm Oil (Packed)", "Gur", "Tea Loose", "Salt Pack (Iodised)", "Potato", "Onion", "Tomato"]

In [8]:
# Create a dictionary to store the last known data for each state
last_known_data = {}
for state in df['States/UTs'].unique():
    last_known_data[state] = df[df['States/UTs'] == state].iloc[-1][crops].values

In [9]:
# Prepare data for LSTM
data = df[crops].values

In [10]:
# Scale the data to be between 0 and 1
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

In [11]:
# Split the data into training and testing sets
"""train_size = int(len(scaled_data) * 0.8)    # 80% of the data is used for training
train_data = scaled_data[0:train_size,:]    #
test_data = scaled_data[train_size:len(scaled_data),:]"""

train_data, test_data = train_test_split(scaled_data, test_size=0.2)

In [12]:
# Create the function to create the dataset
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), :]
        X.append(a)
        Y.append(dataset[i + look_back, :])
    return np.array(X), np.array(Y)

In [13]:
# Set look_back period (number of previous days to consider)
look_back = 7
X_train, Y_train = create_dataset(train_data, look_back)
X_test, Y_test = create_dataset(test_data, look_back)

In [ ]:
# Reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2]))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], X_test.shape[2]))

In [ ]:
# Create and fit the LSTM network
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=50))
model.add(Dense(X_train.shape[2]))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, Y_train, epochs=100, batch_size=1, verbose=2)


In [ ]:
# Make prediction for 6/9/2024
# First, prepare the input data for 6/9/2024 prediction
last_week_data = df[crops].iloc[-look_back:].values
scaled_last_week = scaler.transform(last_week_data)
reshaped_last_week = np.reshape(scaled_last_week, (1, look_back, len(crops)))


In [ ]:
# Predict
prediction_scaled = model.predict(reshaped_last_week)
prediction = scaler.inverse_transform(prediction_scaled)

In [ ]:
# Create DataFrame for 6/9/2024 predictions
predictions_6_9_2024 = pd.DataFrame(data=prediction, columns=crops, index=['Average Price'])

In [ ]:
# Predict for all states using the last known data
for state, state_data in last_known_data.items():
    scaled_state_data = scaler.transform(state_data.reshape(1,-1))
    reshaped_state_data = np.reshape(scaled_state_data, (1, 1, len(crops)))

    state_prediction_scaled = model.predict(reshaped_state_data)
    state_prediction = scaler.inverse_transform(state_prediction_scaled)

    predictions_6_9_2024.loc[state] = state_prediction[0]

In [ ]:
# Add other metrics (Minimum, Maximum, Modal) based on 6/9/2024 'Average Price' prediction
# Note: Modal price prediction is simplistic and could be improved with a more robust method
predictions_6_9_2024.loc['Maximum Price'] = predictions_6_9_2024.loc['Average Price'] * 1.1
predictions_6_9_2024.loc['Minimum Price'] = predictions_6_9_2024.loc['Average Price'] * 0.9
predictions_6_9_2024.loc['Modal Price']  = predictions_6_9_2024.loc['Average Price'].round()


In [ ]:
# Display the predictions
print(predictions_6_9_2024)


In [ ]:
model.save('crop_price_lstm_model.h5')

In [ ]:
model = tensorflow.keras.models.load_model('crop_price_lstm_model.h5')